In [27]:
import re
import time
import datetime
from crawling import *

In [28]:
def geturl(parsed_html):
    result = []
    elements = parsed_html.select("tbody > tr > td:nth-child(1) > a:nth-child(1)")
    for element in elements:
        result.append("https://namu.wiki" + element.attrs.get("href"))
    return result

def gettime(parsed_html):
    alltimes = parsed_html.select("tr time")
    lasttime = datetime.datetime(*map(int, re.split("[-: ]+", alltimes[0].text)))
    firsttime = datetime.datetime(*map(int, re.split("[-: ]+", alltimes[-1].text)))
    return (lasttime - firsttime).seconds//2

In [32]:
page_urls = {}
origin_url = r"https://namu.wiki/RecentChanges"
# get urls until input limit
url_limit = int(input())
while True:
    namu_recent = html_parser(origin_url, 5)
    if namu_recent is not None:
        # get url
        for url in geturl(namu_recent):
            page_urls[url] = page_urls.get(url, 0) + 1
        print("url number:", len(page_urls))
        if len(page_urls) > url_limit:
            break
        # get time dif
        timedif = gettime(namu_recent)
        print(f"waiting time: {timedif}s")
        time.sleep(timedif)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\YJ\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


url number: 78


In [33]:
cnt = 0
titles,  categories ,paragraphs = [],[],[]
for target_url in page_urls:
    if cnt >= 5:
        break
    cnt += 1
    namu_keyword = html_parser(target_url)
    if namu_keyword is not None:
        article = namu_keyword.select_one("article")
        title = get_text(article.select_one("h1"))
        raw_categories = list(article.select_one("h2").next_siblings)[-1].select("li")
        category = "&".join(get_text(raw_categories))
        # remove unnecessary info
        for i in article.select("table"):         
            i.clear()
        # get paragraph data
        if article.select_one(".wiki-heading-content") is None:
            raw_paragraphs = article.select(".wiki-paragraph")
        else:
            raw_paragraphs = article.select(".wiki-heading-content>.wiki-paragraph")
        paragraph = "&".join(get_text(raw_paragraphs))
        titles.append(title)
        categories.append(category)
        paragraphs.append(paragraph)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\YJ\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\YJ\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\YJ\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\YJ\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome vers

In [45]:
import pandas
df = pandas.DataFrame(data={"title":titles, "category":categories, "paragraph":paragraphs})
df

,title,category,paragraph
0,말해줄 수 없는 이야기의 모험,역전재판 시리즈/에피소드,&연이은 무죄 판결을 통해 런던에서 변호사로서 자리 잡은 나루호도. 하지만 이방인인...
1,제13대 국회의원 선거/지역구 국회의원/부산직할시,대한민국 국회의원 선거&제6공화국&1988년 선거,본 항목은 제13대 국회의원 선거의 부산직할시 지역구 선거 결과를 정리하는 항목이다...
2,도화가,로스트아크/클래스,&&2022년 1월 12일 업데이트로 추가될 첫번째 스페셜리스트 클래스이자 요즈족 ...
3,바시티,바시티,2017년 1월 2일 데뷔한 CSO 엔터테인먼트[2]소속의 7인조 그룹이다. 팀명은...
4,밀라보레아스,고룡종&초대형 몬스터&몬스터 헌터 시리즈/등장 생물/현대륙,몬스터 헌터 시리즈에 등장하는 몬스터. 시리즈 첫 작품인 《몬스터 헌터》에서 첫 등...


In [12]:
import re

def text_cleaning(text):
    return re.sub("[^가-힣 ]+", "", text)

In [35]:
df["title"] = df["title"].apply(text_cleaning)
df["category"] = df["category"].apply(text_cleaning)
df["paragraph"] = df["paragraph"].apply(text_cleaning)

In [44]:
from konlpy.tag import *
getmorps = lambda x:x().morphs("가입 후 일이 지나야 편집 가능한 문서입니다")
for i in (Hannanum, Kkma, Komoran, Okt):
    print(getmorps(i))

['아버지가방에들어가', '이', '시ㄴ다']
['아버지', '가방', '에', '들어가', '시', 'ㄴ다']
['아버지', '가방', '에', '들어가', '시', 'ㄴ다']
['아버지', '가방', '에', '들어가신다']


In [69]:
from collections import Counter

all_sentence = "".join(map(str, df["paragraph"]))
sentence = text_cleaning(all_sentence)
# def prenouns(counter):
counter = Counter(Okt().nouns(sentence))
# counter = Counter(Kkma().nouns(sentence))
precounter = Counter({i:counter[i] for i in counter if len(i)>1 and counter[i]>1})
print(precounter)

Counter({'나루': 203, '호도': 203, '이전': 170, '문서': 167, '흑룡': 162, '문단': 159, '역사': 153, '그레이': 122, '때문': 112, '스킬': 108, '헌터': 101, '보레아스': 98, '홍룡': 95, '몬스터': 94, '홈즈': 93, '무기': 83, '공격': 81, '사건': 77, '패턴': 76, '사용': 73, '경우': 72, '아이리스': 68, '브레스': 68, '등장': 66, '직스': 66, '존재': 57, '전당포': 53, '다른': 53, '머리': 52, '자신': 51, '검사': 51, '이후': 50, '재판': 50, '형사': 50, '기술': 48, '퀘스트': 48, '증언': 47, '스사토': 46, '정도': 46, '방어': 46, '파괴': 46, '그렉': 45, '디스크': 44, '속성': 43, '지나': 42, '생각': 41, '사실': 40, '하나': 40, '상황': 39, '상태': 39, '소재': 39, '도화': 38, '시간': 37, '조사': 37, '정보': 36, '장비': 36, '토벌': 36, '시리즈': 35, '또한': 35, '개체': 35, '위해': 34, '매우': 34, '이야기': 32, '내용': 31, '사진': 31, '이상': 31, '트리': 31, '레이드': 31, '사람': 30, '틴피': 30, '모두': 30, '추가': 30, '체력': 30, '거래': 29, '경직': 29, '최종': 28, '주장': 28, '보스': 27, '시작': 27, '이름': 27, '선조': 27, '다만': 26, '선조룡': 26, '모든': 25, '증거': 25, '가의': 25, '형제': 25, '모션': 25, '성능': 25, '부위': 25, '알바': 25, '드래곤': 25, '대해': 24, '세트': 24, '육질': 24, '그렉슨': 23, '접기

In [58]:
korean_stopwords_path = "korean_stopwords.txt"
with open(korean_stopwords_path, encoding="utf8") as f:
    stopwords = list(map(lambda x:x.strip(), f.readlines()))
print(stopwords)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',